# Single analysis notebook

This notebook aims to create enviroment to simply test strategies developed in Stocks & Loss system for python

In [1]:
# imports 
from scraper import stock_daily
from analyzer import Analyzer
from IPython.display import display
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import indicators as ind
import warnings

warnings.filterwarnings('ignore')

## Settings
* Select stock ticker
* Define buy and sell strategy (buystrat and sellstrat)
* Define if you want to run stop loss and what value
* Same with profit taker

In [2]:
xls = pd.ExcelFile('TickerRussel2000.xls')
# stop loss
sl = True
slval = 0.04
# profit taker
pt = False
ptval = 0.2
# repeteated buys
rb = False

## Running calculation

In [3]:
chosen = pd.DataFrame()
fundTickers = pd.DataFrame()
Tickers = pd.DataFrame()
fundTickers["Fund/sector name"] = xls.sheet_names
# print(fundTickers)
# Tickers = pd.DataFrame(,columns=funTickers["Index fund name"])
for i in range(len(xls.sheet_names)):
    sheet1 = pd.read_excel(xls, i)
    fundTickers.loc[i,["Fund/sector ticker"]] = sheet1.columns[1]
    Tickers[fundTickers.loc[i,["Fund/sector name"]]] = sheet1[sheet1.columns[1]][1:]
pd.set_option('display.expand_frame_repr', False)    
display(Tickers)
# I have the initialization here on purpose, to measure its impact
col = ["Ticker", "Trades", "Good trades", "Profit[%]", "Profit[$]", "Hold[%]"]
results = [None] * Tickers.columns.shape[0]
for k in range(Tickers.columns.shape[0]):
    # when I tried to initialize results as [dataframe] * 10 it created list of identical dataframes and when i changed one, it changed them all. Interesting
    results[k] = pd.DataFrame(data=np.zeros([Tickers.shape[0], len(col)]),columns=col)
# iterate over columns
i = 0 
# print(results)
for label, tic in Tickers.iteritems():
    # print(tic)
    # iterate over items in sectors / indexes
    for j, val in tic.iteritems():
        # j starts at 1, instead of usual 0 => reason why there is -1 everywhere
        # analyze stock
        try:
            stock = stock_daily(val, save=False)
            trades = Analyzer(ticker=val,data=stock.data)
            supportLevels = trades.supportLevels()
            resistanceLevels = trades.resistanceLevels()
            smaLine=ind.sma(stock.data,200,price="Close")["SMA"]
            # print(supportLevels[0])
            for i in range(len(supportLevels)):
                #check for similar values
                # print(i)
                # print(supportLevels[0:i+1])
                # print(supportLevels[0:i])
                # print(supportLevels[i-1]-supportLevels[i])
                if i==0:
                    if (stock.data["Close"].iat[-1]-supportLevels[i])/supportLevels[i]<0.02 and stock.data["Close"].iat[-1]-supportLevels[i]>0 and (stock.data["Close"].iat[-1]-smaLine.iat[-1])>0:
                        chosen = chosen.append({"Level type": 'support',"Value": supportLevels[i],"Ticker": stock.ticker, "Current": stock.data["Close"].iat[-1]}, ignore_index=True)    
                elif i>0 and all(abs(supportLevels[0:i]-supportLevels[i])/supportLevels[i]>0.005):
                    if (stock.data["Close"].iat[-1]-supportLevels[i])/supportLevels[i]<0.02 and stock.data["Close"].iat[-1]-supportLevels[i]>0 and (stock.data["Close"].iat[-1]-smaLine.iat[-1])>0:
                        chosen = chosen.append({"Level type": 'support',"Value": supportLevels[i],"Ticker": stock.ticker, "Current": stock.data["Close"].iat[-1]}, ignore_index=True)
            for i in range(len(resistanceLevels)):
                if i==0:
                    if (stock.data["Close"].iat[-1]-resistanceLevels[i])/resistanceLevels[i]<0.02 and stock.data["Close"].iat[-1]-resistanceLevels[i]>0 and (stock.data["Close"].iat[-1]-smaLine.iat[-1])>0:
                        chosen = chosen.append({"Level type": 'resistance',"Value": resistanceLevels[i],"Ticker": stock.ticker, "Current": stock.data["Close"].iat[-1]}, ignore_index=True) 
                elif i>0 and all(abs(resistanceLevels[0:i]-resistanceLevels[i])/resistanceLevels[i]>0.005):
                    if (stock.data["Close"].iat[-1]-resistanceLevels[i])/resistanceLevels[i]<0.02 and stock.data["Close"].iat[-1]-resistanceLevels[i]>0 and (stock.data["Close"].iat[-1]-smaLine.iat[-1])>0:
                        chosen = chosen.append({"Level type": 'resistance',"Value": resistanceLevels[i],"Ticker": stock.ticker, "Current": stock.data["Close"].iat[-1]}, ignore_index=True) 
        except:
            print('stock not found')                      

,Sheet1
1,AMC
2,NTLA
3,CROX
4,LSCC
5,TTEK
...,...
1980,DMS
1981,GBL
1982,WLLBW
1983,PDLI


- XTSLA: No data found, symbol may be delisted
- JWA: No data found, symbol may be delisted
- MOGA: No data found, symbol may be delisted
- LGFB: No data found, symbol may be delisted
- CWENA: No data found, symbol may be delisted
- PMVP: CircuitBreaker 'redis' is OPEN and does not permit further calls
- KIDS: CircuitBreaker 'redis' is OPEN and does not permit further calls
- SPNS: CircuitBreaker 'redis' is OPEN and does not permit further calls
- VVI: CircuitBreaker 'redis' is OPEN and does not permit further calls
- LGFA: No data found, symbol may be delisted
- MSFUT: No data found, symbol may be delisted
- ETD: No data found for this date range, symbol may be delisted
- GEFB: No data found, symbol may be delisted
stock not found
- CRDA: No data found for this date range, symbol may be delisted
- VIA: No data found for this date range, symbol may be delisted
- -: No data found, symbol may be delisted
- ADRO: No data found, symbol may be delisted
- PDLI: No data found, symbol may be d

In [4]:
# print(chosen)
chosen.to_excel('suportyRussel.xlsx')
